In [6]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf #tensorflow==1.13.1
np.random.seed(1337)

'''this code is from https://www.guru99.com/artificial-neural-network-tutorial.html'''
with open("./pickled_mnist.pkl", "br") as fh:
    data = pickle.load(fh)

X_train = data[0].astype(np.int) #train_images
X_test = data[1].astype(np.int) #test_images
y_train = data[2].astype(np.int) #train_labels
y_test = data[3].astype(np.int) #test_labels
batch_size =len(X_train)

class_names = ['small', 'medium', 'large']

'''
print(train_images.shape) #(1619, 172800)
print(test_images.shape) #(460, 172800)
print(train_labels.shape) #(1619, 3)
print(test_labels.shape) #(460, 3)
'''
train_images=X_train.reshape(1619, 240, 240, 3)
test_images=X_test.reshape(460, 240, 240, 3)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1619, 172800) (1619, 1) (460, 172800) (460, 1)


In [7]:
## resclae
from sklearn.preprocessing import MinMaxScaler #scikit-learn 0.22.2
scaler = MinMaxScaler()
# Train
X_train_scaled = scaler.fit_transform(X_train.astype(np.int))
# test
X_test_scaled = scaler.fit_transform(X_test.astype(np.int))
feature_columns = [tf.feature_column.numeric_column('x', shape=X_train_scaled.shape[1:])]



estimator = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[300, 100], 
    n_classes=3, 
    model_dir = './train/DNN')
    
# Train the estimator
train_input = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train_scaled},
    y=y_train,
    batch_size=50,
    shuffle=False,
    num_epochs=None)
estimator.train(input_fn = train_input,steps=1000) 
eval_input = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test_scaled},
    y=y_test, 
    shuffle=False,
    batch_size=X_test_scaled.shape[0],
    num_epochs=1)
estimator.evaluate(eval_input,steps=None) 


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './train/DNN', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000018F533722E8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was

{'accuracy': 0.8108696,
 'average_loss': 0.67764705,
 'loss': 311.71765,
 'global_step': 1000}

In [ ]:
#improve the model
estimator_imp = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[300, 100],
    dropout=0.3, 
    n_classes = 10,
    optimizer=tf.train.ProximalAdagradOptimizer(
      learning_rate=0.01,
      l1_regularization_strength=0.01, 
      l2_regularization_strength=0.01
    ),
    model_dir = '/train/DNN1')
estimator_imp.train(input_fn = train_input,steps=1000) 
estimator_imp.evaluate(eval_input,steps=None) 